In [4]:
import numpy as np
import pandas as pd
import json
import requests
import re
import os
from datetime import datetime as dt
from pandas.io.json import json_normalize
import geocoder
import matplotlib.pylab as plt
import seaborn as sns
import shapefile
from scipy import stats
import math
import statistics
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import normalize


# Split datasets into train and test datasets
%matplotlib inline

df_final = pd.read_csv('dat_final.csv')
y = df_final['price_sqm']
X = df_final.drop(['price_sqm'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.25, random_state = 2020)

#### Scale the data

In [5]:
scaler = StandardScaler().fit(X_train)
X_trains = scaler.transform(X_train)
X_tests = scaler.transform(X_test)

#### Lasso

In [7]:
lasso = Lasso()
param = {'alpha':[0.001, 0.01, 0.1, 0.2, 0.5, 0.7, 1, 1.5, 2]}

lasso_cv = GridSearchCV(lasso,
                       param,
                       scoring = 'neg_mean_squared_error',
                       cv = 5,
                       verbose = 0)

In [8]:
lasso_cv.fit(X_trains,y_train)

/Users/cj/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 59312375804.98064, tolerance: 83496832.6994322
  model = cd_fast.enet_coordinate_descent(
/Users/cj/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 64388995714.5348, tolerance: 83674583.44662924
  model = cd_fast.enet_coordinate_descent(
/Users/cj/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 76786671020.4771, tolerance: 83547214.78608872
  model = cd_fast.enet_coordinate_descent(
/Users/cj/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:5

GridSearchCV(cv=5, estimator=Lasso(),
             param_grid={'alpha': [0.001, 0.01, 0.1, 0.2, 0.5, 0.7, 1, 1.5, 2]},
             scoring='neg_mean_squared_error')

In [9]:
best_alpha = pd.Series(lasso_cv.best_params_)[0]

print(lasso_cv.best_score_)
print(lasso_cv.best_params_)

-474217.8289708067
{'alpha': 0.001}


In [10]:
lasso = Lasso(alpha = best_alpha, 
              random_state = 2020)
lasso = lasso.fit(X_trains, y_train)

/Users/cj/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 92630819174.69376, tolerance: 104434646.31011233
  model = cd_fast.enet_coordinate_descent(


In [12]:
yhat = lasso.predict(X_trains)
MAE_lasso = round(mean_absolute_error(y_train, yhat),2)
RMSE_lasso = round(math.sqrt(mean_squared_error(y_train, yhat)),2)
R2_lasso = round(r2_score(y_train, yhat),2)


print('-------------- Lasso Regression Model on training set------------')
print('MAE: ' + str(mean_absolute_error(y_train, yhat)))
print('RMSE: '+ str(math.sqrt(mean_squared_error(y_train, yhat))))
print('R^2 Score: ' + str(r2_score(y_train, yhat)))

-------------- Lasso Regression Model on training set------------
MAE: 545.0195911020379
RMSE: 688.5416471044475
R^2 Score: 0.7141393683346489


In [13]:
yhat = lasso.predict(X_tests)
MAE_lasso = round(mean_absolute_error(y_test, yhat),2)
RMSE_lasso = round(math.sqrt(mean_squared_error(y_test, yhat)),2)
R2_lasso = round(r2_score(y_test, yhat),2)


print('-------------- Lasso Regression Model ------------')
print('MAE: ' + str(mean_absolute_error(y_test, yhat)))
print('RMSE: '+ str(math.sqrt(mean_squared_error(y_test, yhat))))
print('R^2 Score: ' + str(r2_score(y_test, yhat)))




-------------- Lasso Regression Model ------------
MAE: 546.5311695891602
RMSE: 690.5956323677818
R^2 Score: 0.7143367664217825


In [6]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, RepeatedKFold

# Convert datasets to XGBDMatrix
dtrain = xgb.DMatrix(X_trains, label = y_train)
dtest = xgb.DMatrix(X_tests)


In [7]:
# Define model parameters
xgb_params = {'obj':'reg:squarederror',
              'eval_metric':'rmse',
              'max_depth':6, 
              'eta':0.5,
              'min_child_weight':1,
              'subsample':1,
              'colsample_bytree':1}

# Find model parameters 'num_boost_round' using cv
cv_res = xgb.cv(xgb_params, 
                dtrain, 
                num_boost_round=500, 
                nfold=5, 
                seed=2020, 
                stratified=False,
                early_stopping_rounds=25, 
                verbose_eval=10, 
                show_stdv=True)

# Get best num_boost_round
nrounds = cv_res[cv_res['test-rmse-mean'] == cv_res['test-rmse-mean'].min()].index[0]
cv_res[cv_res['test-rmse-mean'] == cv_res['test-rmse-mean'].min()]

[21:54:23] WARNING: /Users/cj/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.8/xgboost/src/learner.cc:548: 
Parameters: { obj } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[21:54:27] WARNING: /Users/cj/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.8/xgboost/src/learner.cc:548: 
Parameters: { obj } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[21:54:31] WARNING: /Users/cj/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.8/xgboost/src/learner.cc:548: 
Parameters: { obj } might not be used.

  This may not be accurate due to some parameters are 

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
499,169.625684,0.27956,192.434189,0.508204


In [8]:
xgb_def = xgb.train(params = xgb_params,
                   num_boost_round = nrounds,
                   dtrain = dtrain)

[00:12:32] WARNING: /Users/cj/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.8/xgboost/src/learner.cc:548: 
Parameters: { obj } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [9]:
yhat = xgb_def.predict(dtest)
print('-------------- XGB Model: Baseline Parameters ------------')
print('MAE: ' + str(mean_absolute_error(y_test, yhat)))
print('RMSE: '+ str(math.sqrt(mean_squared_error(y_test, yhat))))
print('R^2 Score: ' + str(r2_score(y_test, yhat)))

-------------- XGB Model: Baseline Parameters ------------
MAE: 137.90706216763843
RMSE: 190.71381162168254
R^2 Score: 0.9782143267633299
